In [1]:
%pip install opendatasets as 
%pip install sklearn

ERROR: Could not find a version that satisfies the requirement as (from versions: none)
ERROR: No matching distribution found for as
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


import opendatasets as od
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [13]:
#  Download dataset
od.download("https://www.kaggle.com/competitions/goodreads-books-reviews-290312/data")

Skipping, found downloaded files in "./goodreads-books-reviews-290312" (use force=True to force download)


In [3]:
df = pd.read_csv("./goodreads-books-reviews-290312/goodreads_train.csv")
dft = pd.read_csv("./goodreads-books-reviews-290312/goodreads_test.csv")

df[['date_added','date_updated','read_at','started_at']] = df[['date_added','date_updated','read_at','started_at']].apply(pd.to_datetime)
df ['review_length'] = df['review_text'].str.len()
dft ['review_length'] = dft['review_text'].str.len()

In [4]:

df['read_duration'] = (df['read_at'] - df['started_at']).dt.days.fillna(0)

df.head()
df.size

,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments,review_length,read_duration
0,8842281e1d1347389f2ab93d60773d4d,18245960,dfdbb7b0eb5a7e4c26d59a937e2e5feb,5,This is a special book. It started slow for ab...,2017-07-30 07:44:10-07:00,2017-08-30 00:00:26-07:00,2017-08-26 12:05:52-07:00,2017-08-15 13:23:18-07:00,28,1,2119,10.0
1,8842281e1d1347389f2ab93d60773d4d,16981,a5d2c3628987712d0e05c4f90798eb67,3,Recommended by Don Katz. Avail for free in Dec...,2016-12-05 10:46:44-08:00,2017-03-22 11:37:04-07:00,NaN,NaN,1,0,93,0.0
2,8842281e1d1347389f2ab93d60773d4d,28684704,2ede853b14dc4583f96cf5d120af636f,3,"A fun, fast paced science fiction thriller. I ...",2016-11-15 11:29:22-08:00,2017-03-20 23:40:27-07:00,2017-03-18 23:22:42-07:00,2017-03-17 23:45:40-07:00,22,0,2598,0.0
3,8842281e1d1347389f2ab93d60773d4d,27161156,ced5675e55cd9d38a524743f5c40996e,0,Recommended reading to understand what is goin...,2016-11-09 17:37:04-08:00,2016-11-09 17:38:20-08:00,NaN,NaN,5,1,153,0.0
4,8842281e1d1347389f2ab93d60773d4d,25884323,332732725863131279a8e345b63ac33e,4,"I really enjoyed this book, and there is a lot...",2016-04-25 09:31:23-07:00,2016-04-25 09:31:23-07:00,2016-06-26 00:00:00-07:00,2016-05-28 00:00:00-07:00,9,1,4044,29.0


11700000

In [5]:
Y = df['rating']
X = df[['n_votes','n_comments','review_length','read_duration']]

In [6]:
from sklearn import linear_model
regr = linear_model.LinearRegression()
regr.fit(X,Y)

LinearRegression()

In [46]:
import statsmodels.api as sm
X = sm.add_constant(X)
model = sm.OLS(Y,X).fit()
model.summary()  

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 rating   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     651.3
Date:                Fri, 05 Aug 2022   Prob (F-statistic):               0.00
Time:                        09:50:17   Log-Likelihood:            -1.4784e+06
No. Observations:              900000   AIC:                         2.957e+06
Df Residuals:                  899995   BIC:                         2.957e+06
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             3.6341      0.002   2089.774      0.000       3.631       3.638
n_votes           0.0008   9.48e-05      8.050      0.000       0.001       0.001
n_comments       -0.0041      0.000    -14.275      0.000      -0.005      -0.004
review_length  5.304e-05   1.07e-06     49.551      0.000    5.09e-05    5.51e-05
read_duration  9.011e-05   2.03e-05      4.450      0.000    5.04e-05       0.000
==============================================================================
Omnibus:                   132006.090   Durbin-Watson:                   1.553
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           200088.853
Skew:                          -1.072   Prob(JB):                         0.00
Kurtosis:                       3.862   Cond. No.                     2.17e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.17e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [7]:
from sklearn import tree
clf = tree.DecisionTreeClassifier(max_depth=2,random_state=0)
clf = clf.fit(X, Y)
dot_data = tree.export_graphviz(clf,
                     out_file=None,
                     filled = True)
graph = graphviz.Source(dot_data)
graph

NameError: name 'graphviz' is not defined